In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('ny_taxi_source_data/yellow_tripdata_2015-01.csv', nrows=500)

In [3]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30


### Use Pnadas to Create an SQL schema for the dataframe

In [4]:
print(pd.io.sql.get_schema(df, name='yellow_taxi_2015_01'))

CREATE TABLE "yellow_taxi_2015_01" (
"VendorID" INTEGER,
  "tpep_pickup_datetime" TEXT,
  "tpep_dropoff_datetime" TEXT,
  "passenger_count" INTEGER,
  "trip_distance" REAL,
  "pickup_longitude" REAL,
  "pickup_latitude" REAL,
  "RateCodeID" INTEGER,
  "store_and_fwd_flag" TEXT,
  "dropoff_longitude" REAL,
  "dropoff_latitude" REAL,
  "payment_type" INTEGER,
  "fare_amount" REAL,
  "extra" REAL,
  "mta_tax" REAL,
  "tip_amount" REAL,
  "tolls_amount" REAL,
  "improvement_surcharge" REAL,
  "total_amount" REAL
)


Change the data type for the pickup and dropoff datetime:

In [5]:
df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])

### Using SQL Alchemy to Load the Data to Postgres

In [6]:
from sqlalchemy import create_engine

In [7]:
# Define the database connection parameters
db_params = {
    'host': 'localhost',
    'database': 'ny_taxi',
    'user': 'postgres',
    'password': '358',
    'port': '5433'  # localhost port 
}

# Create a SQLAlchemy engine
engine = create_engine(f'postgresql://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["database"]}')

In [8]:
# check if the connection pull through
engine.connect()

In [9]:
print(pd.io.sql.get_schema(df, name='yellow_taxi_2015_01', con=engine))


CREATE TABLE yellow_taxi_2015_01 (
	"VendorID" BIGINT, 
	tpep_pickup_datetime TIMESTAMP WITHOUT TIME ZONE, 
	tpep_dropoff_datetime TIMESTAMP WITHOUT TIME ZONE, 
	passenger_count BIGINT, 
	trip_distance FLOAT(53), 
	pickup_longitude FLOAT(53), 
	pickup_latitude FLOAT(53), 
	"RateCodeID" BIGINT, 
	store_and_fwd_flag TEXT, 
	dropoff_longitude FLOAT(53), 
	dropoff_latitude FLOAT(53), 
	payment_type BIGINT, 
	fare_amount FLOAT(53), 
	extra FLOAT(53), 
	mta_tax FLOAT(53), 
	tip_amount FLOAT(53), 
	tolls_amount FLOAT(53), 
	improvement_surcharge FLOAT(53), 
	total_amount FLOAT(53)
)




We will ise the schema above to create schema for the postgres database. Also we will load the data in chunks using pandas since it is very large (Over a milinon records).

In [20]:
# the result of loading the data in chunks is an iterator
iter_df = pd.read_csv('ny_taxi_source_data/yellow_tripdata_2015-01.csv', chunksize=100000, iterator=True)

In [21]:
# for every next, the data will return a dataframe of 100000 records
df_100k = next(iter_df)

In [22]:
df_100k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               100000 non-null  int64  
 1   tpep_pickup_datetime   100000 non-null  object 
 2   tpep_dropoff_datetime  100000 non-null  object 
 3   passenger_count        100000 non-null  int64  
 4   trip_distance          100000 non-null  float64
 5   pickup_longitude       100000 non-null  float64
 6   pickup_latitude        100000 non-null  float64
 7   RateCodeID             100000 non-null  int64  
 8   store_and_fwd_flag     100000 non-null  object 
 9   dropoff_longitude      100000 non-null  float64
 10  dropoff_latitude       100000 non-null  float64
 11  payment_type           100000 non-null  int64  
 12  fare_amount            100000 non-null  float64
 13  extra                  100000 non-null  float64
 14  mta_tax                100000 non-nul

We will create a tabsle in the postgres database using the dataframe before loading the data in chunks into it. This will use the schema of the dataframe to create the schema for the table in postgres:

In [25]:
df_100k["tpep_pickup_datetime"] = pd.to_datetime(df_100k["tpep_pickup_datetime"])
df_100k["tpep_dropoff_datetime"] = pd.to_datetime(df_100k["tpep_dropoff_datetime"])

In [26]:
df_100k.head(0).to_sql(name='yellow_taxi_2015_01', con=engine, if_exists='replace') # if the table name exists, drop and replace it

0